<a href="https://colab.research.google.com/github/iiCellxx/CSST-101-ALMARIO/blob/main/LSTM_Practice_Chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install tensorflow


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import pandas as pd
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

In [4]:
# List of Excel file paths
excel_files = [
    '/content/drive/MyDrive/rotcChatbot/ROTC Knowledge Excel Files/Military-Discipline-Customs-and-Courtesy.xlsx',
    '/content/drive/MyDrive/rotcChatbot/ROTC Knowledge Excel Files/Military-Justice-System.xlsx',
    '/content/drive/MyDrive/rotcChatbot/ROTC Knowledge Excel Files/Military-Values-Ethics-and-Professionalism.xlsx',
    '/content/drive/MyDrive/rotcChatbot/ROTC Knowledge Excel Files/MilitaryLeadership.xlsx',
    '/content/drive/MyDrive/rotcChatbot/ROTC Knowledge Excel Files/Philippine Military History.xlsx',
    '/content/drive/MyDrive/rotcChatbot/ROTC Knowledge Excel Files/Threats-To-National-Security.xlsx'
]

# Combine all Excel files into a single JSON file
all_data = []
for file in excel_files:
    df = pd.read_excel(file)
    all_data.append(df)

merged_data = pd.concat(all_data, ignore_index=True)
json_file = '/content/drive/MyDrive/rotcChatbot/rotc_data.json'
merged_data.to_json(json_file, orient="records", indent=4)
print(f"Excel files successfully converted to JSON and saved as {json_file}")


Excel files successfully converted to JSON and saved as /content/drive/MyDrive/rotcChatbot/rotc_data.json


In [29]:
import nltk
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np

# Load the JSON file
combined_df = pd.read_json(json_file)

# Ensure the required columns are present
required_columns = ['Tags', 'Pattern', 'Response', 'Context']
if not all(col in combined_df.columns for col in required_columns):
    print(f"Error: JSON file must include these columns: {required_columns}")
else:
    print(f"Loaded {len(combined_df)} entries with Tags, Pattern, Response, and Context.")

# Download NLTK punkt tokenizer
nltk.download('punkt')

# Combine 'tags' and 'patterns' as the input and 'responses' with 'context' as the output
combined_df['input_text'] = combined_df['Tags'] + ": " + combined_df['Pattern']
combined_df['output_text'] = combined_df['Response'] + " " + combined_df['Context']

# Tokenize and pad the input text
tokenizer = Tokenizer(oov_token="<OOV>")
tokenizer.fit_on_texts(combined_df['input_text'])

# Convert input text to sequences
X = tokenizer.texts_to_sequences(combined_df['input_text'])

# Pad input sequences to ensure uniform input length
max_input_len = max(len(seq) for seq in X)
X_pad = pad_sequences(X, maxlen=max_input_len, padding='post')

# Encode the output text (e.g., responses and context)
output_tokenizer = Tokenizer(oov_token="<OOV>")
output_tokenizer.fit_on_texts(combined_df['output_text'])
y = output_tokenizer.texts_to_sequences(combined_df['output_text'])

# Pad output sequences
max_output_len = max(len(seq) for seq in y)
y_pad = pad_sequences(y, maxlen=max_output_len, padding='post')

# Show processed input and output
print("Tokenized and padded input (first 5 examples):")
print(X_pad[:5])
print("Tokenized and padded output (first 5 examples):")
print(y_pad[:5])


Loaded 247 entries with Tags, Pattern, Response, and Context.
Tokenized and padded input (first 5 examples):
[[  4  56   3   7   2  42   6   4  56   0   0   0   0   0   0   0]
 [ 57   8  10   2   4  57  31   2   4 168   0   0   0   0   0   0]
 [ 58  59  95  60  58  59  29 361   5   2  12   0   0   0   0   0]
 [  4  61   3  13   2 362  24 169 363   5   2   4   0   0   0   0]
 [  4  62  23  13  62  25   5   4  48   0   0   0   0   0   0   0]]
Tokenized and padded output (first 5 examples):
[[  3 171   4   5 134  17   6  45 314 450  22   2 315  49   3   5   5 134
   27  22 172 680   2 235  56  39   2  46   0   0   0   0   0   0   0   0
    0]
 [  3 135  17 136   6  56  46   2 173   9 236   4  15   7   5  67 316 451
   28   9 681 682   4  22 683 174   2 317   4  47   0   0   0   0   0   0
    0]
 [175 176 137  62 684  23 112 237 113   2   7   3 238   4 235 175 176 452
  239  87   6 453 177  23 454 685 455   0   0   0   0   0   0   0   0   0
    0]
 [  3 686  11 456  14 318   7   5 319   2 

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [30]:
import nltk
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
from sklearn.preprocessing import LabelEncoder

# Encode the output text (e.g., responses and context)
# Convert responses to categorical labels
label_encoder = LabelEncoder()
y_labels = label_encoder.fit_transform(combined_df['Response'])  # Assuming 'Response' is the target

# Pad input sequences to ensure uniform input length
max_input_len = max(len(seq) for seq in X)
X_pad = pad_sequences(X, maxlen=max_input_len, padding='post')


In [31]:
# Build the LSTM model
model = Sequential()
model.add(Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=32, input_length=X_pad.shape[1]))
model.add(LSTM(64, return_sequences=False))
model.add(Dense(32, activation='relu'))
model.add(Dense(len(label_encoder.classes_), activation='softmax'))  # Output layer with correct number of classes

# Compile the model
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Summary of the model
model.summary()


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_4 (Embedding)              │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_4 (LSTM)                        │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_7 (Dense)                      │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_8 (Dense)                      │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [32]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
import os

# Define checkpoint directory in Google Drive
checkpoint_dir = '/content/drive/MyDrive/rotcChatbot/checkpoints/'
os.makedirs(checkpoint_dir, exist_ok=True)

# Define checkpoint and final model paths
checkpoint_path = os.path.join(checkpoint_dir, 'chatbot_model_checkpoint.keras')
final_model_path = '/content/drive/MyDrive/rotcChatbot/lstm_chatbot_model.keras'

# Set up EarlyStopping and ModelCheckpoint callbacks
early_stopping = EarlyStopping(
    monitor='loss',  # Monitor training loss (you can also monitor 'val_loss' if using validation)
    patience=3,      # Stop training if no improvement after 3 epochs
    restore_best_weights=True  # Restore the model with the best weights
)

checkpoint_callback = ModelCheckpoint(
    filepath=checkpoint_path,  # Save checkpoint to Google Drive
    save_best_only=True,       # Save only the best model based on monitored metric
    monitor='loss',            # Monitor training loss
    verbose=1                  # Print messages when saving checkpoints
)

# Train the model with EarlyStopping and ModelCheckpoint
history = model.fit(
    X_pad, y_labels,  # Use encoded labels for training
    epochs=50,             # Maximum number of epochs
    batch_size=32,          # Batch size
    verbose=1,              # Display progress
    callbacks=[early_stopping, checkpoint_callback]  # Apply callbacks
)

# Save the final trained model to Google Drive
model.save(final_model_path)
print(f"Final model saved at {final_model_path}")


Epoch 1/50
7/8 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.0000e+00 - loss: 5.5103
Epoch 1: loss improved from inf to 5.51184, saving model to /content/drive/MyDrive/rotcChatbot/checkpoints/chatbot_model_checkpoint.keras
8/8 ━━━━━━━━━━━━━━━━━━━━ 3s 16ms/step - accuracy: 0.0000e+00 - loss: 5.5107
Epoch 2/50
7/8 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.0150 - loss: 5.5089 
Epoch 2: loss improved from 5.51184 to 5.50930, saving model to /content/drive/MyDrive/rotcChatbot/checkpoints/chatbot_model_checkpoint.keras
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.0134 - loss: 5.5090
Epoch 3/50
7/8 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.0096 - loss: 5.5090    
Epoch 3: loss improved from 5.50930 to 5.50825, saving model to /content/drive/MyDrive/rotcChatbot/checkpoints/chatbot_model_checkpoint.keras
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 0.0120 - loss: 5.5088
Epoch 4/50
5/8 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.0290 - loss: 5.5063     
Epoch 4: loss

In [33]:
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Function to predict responses
def get_response(text):
    # Convert user input to sequences
    seq = tokenizer.texts_to_sequences([text])
    # Pad the sequence to match the training input shape
    padded = pad_sequences(seq, maxlen=X_pad.shape[1], padding='post')
    # Predict the probabilities for each response
    pred = model.predict(padded, verbose=0)
    # Get the index of the highest probability
    response_idx = np.argmax(pred)
    # Return the corresponding response
    return combined_df['Response'][response_idx]

# Chatbot loop
print("Hello! I'm your ROTC Chatbot. Ask me anything about military topics. Type 'exit' to end the conversation.")
while True:
    user_input = input("You: ")
    if user_input.lower() == 'exit':
        print("Goodbye! Stay sharp!")
        break
    try:
        # Get and print the chatbot's response
        response = get_response(user_input)
        print("Chatbot:", response)
    except Exception as e:
        # Handle any errors (e.g., unrecognized input)
        print("Chatbot: Sorry, I didn't understand that. Please try asking something else.")


Hello! I'm your ROTC Chatbot. Ask me anything about military topics. Type 'exit' to end the conversation.
You: Military Courtesy
Chatbot: Army leaders help develop their teams by encouraging initiative, teamwork, and personal responsibility, while caring for team members.
You: What is the purpose of military courtesy?
Chatbot: Following cadet etiquette helps develop a cadet’s professionalism and respect for authority.
You: What is the purpose of military courtesy?
Chatbot: Following cadet etiquette helps develop a cadet’s professionalism and respect for authority.
You: Salute
Chatbot: Army leaders help develop their teams by encouraging initiative, teamwork, and personal responsibility, while caring for team members.
You: Ranks
Chatbot: Army leaders help develop their teams by encouraging initiative, teamwork, and personal responsibility, while caring for team members.
You: l
Chatbot: Army leaders help develop their teams by encouraging initiative, teamwork, and personal responsibility